# Task 4 — General Health Query Chatbot (Prompt Engineering Based)
#
## Objective:
## - Build a simple chatbot that answers general health-related questions using an LLM.
## - Use prompt engineering to make responses friendly and clear.
## - Add safety filters to avoid giving harmful medical advice.
##
## Notes:
## - This notebook supports OpenAI's Chat API (gpt-3.5) if you set OPENAI_API_KEY as an env var.
## - An optional fallback path using Hugging Face Inference API is provided (requires HF token).
## - Always include disclaimers and refuse to provide diagnosis/medical prescriptions.


In [3]:

# Basic imports
import os
import re
import sys
import json
from getpass import getpass

# Install openai if not available (uncomment in notebook)
# !pip install openai

# %%
# Helper: Safety filter (simple rule-based)
# If a user asks for diagnosis, dosing, or emergency instructions, the assistant will refuse and direct to a professional.
DANGEROUS_PATTERNS = [
    r'\bdiagnos(e|is|ing)\b',
    r'\bprescrib(e|ing|ed)\b',
    r'\bwhat (dose|dosage)\b',
    r'\bhow much (mg|ml|tablets?)\b',
    r'\bemergency\b',
    r'\bpoison\b',
    r'\bself[- ]harm\b',
    r'\bkill myself\b',
    r'\bsuicid(e|al)\b'
]

def is_dangerous_query(text):
    t = text.lower()
    for p in DANGEROUS_PATTERNS:
        if re.search(p, t):
            return True
    return False

# %%
# OpenAI Chat function (if you have an API key)
def chat_with_openai(messages, model="gpt-3.5-turbo", max_tokens=400, temperature=0.2):
    try:
        import openai
    except Exception as e:
        raise RuntimeError("openai package not installed. pip install openai") from e

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OpenAI API key not found. Set OPENAI_API_KEY env var.")
    openai.api_key = api_key

    resp = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    return resp['choices'][0]['message']['content'].strip()

# %%
# Hugging Face Inference fallback (optional) - uses text-generation via inference API
def chat_with_hf(prompt, hf_token=None, model="gpt2"):
    import requests
    if hf_token is None:
        hf_token = os.getenv("HF_API_TOKEN")
    if not hf_token:
        raise RuntimeError("Hugging Face API token not found. Set HF_API_TOKEN env var.")
    headers = {"Authorization": f"Bearer {hf_token}"}
    api_url = f"https://api-inference.huggingface.co/models/{model}"
    payload = {"inputs": prompt, "options": {"wait_for_model": True}}
    resp = requests.post(api_url, headers=headers, json=payload, timeout=120)
    if resp.status_code != 200:
        raise RuntimeError(f"Hugging Face inference failed: {resp.status_code} {resp.text}")
    output = resp.json()
    # output can be list of generations
    try:
        return output[0]['generated_text']
    except Exception:
        return str(output)

# %%
# Prompt engineering: system + few-shot examples
SYSTEM_PROMPT = """
You are a helpful and friendly medical information assistant. Provide general, non-diagnostic health information in clear, empathetic language.
Always include a short disclaimer: "I am not a doctor. For medical diagnosis or treatment, please consult a healthcare professional."
If a query requests diagnosis, prescription, dosing, or emergency/urgent actions, refuse politely and direct the user to seek professional help or emergency services.
Prioritize safety and do not provide specific dosing or prescription instructions.
"""

# Short few-shot examples to guide style
EXAMPLES = [
    {"role":"user", "content":"What causes a sore throat?"},
    {"role":"assistant", "content":"A sore throat can be caused by viral infections (like the common cold), bacterial infections (such as strep throat), allergies, dry air, or irritation from smoke. Rest, fluids, and saltwater gargles can help. If you have a high fever, difficulty breathing, or severe pain, see a healthcare professional."},
    {"role":"user", "content":"Is paracetamol safe for children?"},
    {"role":"assistant", "content":"Paracetamol (acetaminophen) is commonly used for fever and pain in children, but the dose depends on the child's weight and age. Always follow product dosing instructions or ask a pediatrician. If you suspect an overdose or the child is unwell, seek immediate medical attention. I am not a doctor."}
]

# %%
# Interactive loop (works in Jupyter)
def run_chatbot():
    print("General Health Chatbot. Type 'exit' to quit.")
    # Build base messages
    base_messages = [{"role":"system","content":SYSTEM_PROMPT}]
    # Add examples
    for ex in EXAMPLES:
        base_messages.append(ex)

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ("exit", "quit"):
            print("Goodbye — remember this is only general information.")
            break

        # Safety filter
        if is_dangerous_query(user_input):
            print("Assistant: I'm sorry, I can't provide medical diagnoses, prescriptions, or emergency instructions. Please contact a healthcare professional or emergency services.")
            continue

        # Prepare messages
        msgs = base_messages + [{"role":"user", "content": user_input}]
        # Try OpenAI first
        try:
            answer = chat_with_openai(msgs)
        except Exception as e:
            print(f"[OpenAI unavailable or error: {e}] Trying Hugging Face fallback if configured...")
            try:
                prompt = SYSTEM_PROMPT + "\n\nUser: " + user_input + "\nAssistant:"
                answer = chat_with_hf(prompt)
            except Exception as e2:
                print("No LLM backend available:", e2)
                continue

        # Add final safety reminder (will also be in system prompt)
        if "I am not a doctor" not in answer:
            answer = answer.strip() + "\n\nDisclaimer: I am not a doctor. For a medical diagnosis or treatment plan, consult a qualified healthcare professional."

        print("\nAssistant:", answer)

# %%
# Run the interactive chatbot (uncomment to run in a notebook)
# run_chatbot()

# %%
# Example usage programmatically
def answer_query(query, backend="openai"):
    if is_dangerous_query(query):
        return "I'm sorry, I can't help with diagnosis, prescriptions, dosing, or emergency instructions. Please contact a healthcare professional or emergency services."

    msgs = [{"role":"system","content":SYSTEM_PROMPT}]
    for ex in EXAMPLES:
        msgs.append(ex)
    msgs.append({"role":"user","content":query})

    if backend=="openai":
        return chat_with_openai(msgs)
    else:
        prompt = SYSTEM_PROMPT + "\n\nUser: " + query + "\nAssistant:"
        return chat_with_hf(prompt)

# %%
# Quick test (replace with actual API keys in env vars)
if __name__ == "__main__" and sys.stdin.isatty():
    print("Run interactive chatbot by calling run_chatbot() or using answer_query().")